## Import the modules

In [3]:
import pandas as pd
import numpy as np
import os
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input

## Load the dataset and randomize it

In [32]:
BASE_DIR = '../../dataset/'
TRAINING_DIR = '../../training_dataset/'

In [36]:
# labels - age
import random
import shutil
image_paths = []
age_labels = []
age_count = {}

if os.path.exists(TRAINING_DIR):
    shutil.rmtree(TRAINING_DIR)
os.makedirs(TRAINING_DIR, exist_ok=True)

files = os.listdir(BASE_DIR)
random.shuffle(files)

for filename in files:
    image_path = os.path.join(BASE_DIR, filename)
    temp = filename.split('_')
    age = int(temp[0])
    if age_count.get(age, 0) < 244:
        shutil.copy(os.path.join(BASE_DIR, filename), os.path.join(TRAINING_DIR, filename))
        image_paths.append(os.path.join(TRAINING_DIR, filename))
        age_labels.append(age)        
        age_count[age] = age_count.get(age, 0) + 1

In [37]:
# convert to dataframe
df = pd.DataFrame()
df['image'], df['age'] = image_paths, age_labels
df.head()

,image,age
0,../../training_dataset/29_0_2_2017011618220671...,29
1,../../training_dataset/6_1_4_20170103213155772...,6
2,../../training_dataset/18_1_3_2017010921274236...,18
3,../../training_dataset/26_1_2_2017011618085413...,26
4,../../training_dataset/30_0_1_2017011403343458...,30


## Feature Extraction

In [5]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = np.array(img)
        features.append(img)
        
    features = np.array(features)
    features = features.reshape(len(features), 128, 128, 1)
    return features

In [7]:
from PIL import Image
X = extract_features(df['image'])

  0%|          | 0/23700 [00:00<?, ?it/s]

In [8]:
X.shape

(23700, 128, 128, 1)

In [10]:
# normalize
X = X / 255.0

In [14]:
y_age = np.array(df['age'])

In [12]:
input_shape = (128, 128, 1)

In [21]:
inputs = Input((input_shape))
# convolutional layers
conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu') (inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2)) (conv_1)
conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu') (maxp_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2)) (conv_2)
conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu') (maxp_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2)) (conv_3)
conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu') (maxp_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2)) (conv_4)

flatten = Flatten() (maxp_4)

# fully connected layers
dense_2 = Dense(256, activation='relu') (flatten)

dropout_2 = Dropout(0.3) (dense_2)

output_2 = Dense(1, activation='relu', name='age_out') (dropout_2)

model = Model(inputs=[inputs], outputs=[output_2])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [22]:
# train model
history = model.fit(x=X, y=y_age, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
593/593 [==============================] - 156s 260ms/step - loss: 182.8489 - mae: 10.3640 - val_loss: 1683.7228 - val_mae: 36.8046
Epoch 2/30
593/593 [==============================] - 149s 250ms/step - loss: 107.5335 - mae: 7.9427 - val_loss: 978.5715 - val_mae: 28.0940
Epoch 3/30
593/593 [==============================] - 148s 250ms/step - loss: 86.0203 - mae: 6.9840 - val_loss: 891.2847 - val_mae: 26.5544
Epoch 4/30
593/593 [==============================] - 149s 251ms/step - loss: 75.7325 - mae: 6.5190 - val_loss: 905.8669 - val_mae: 26.8741
Epoch 5/30
593/593 [==============================] - 149s 251ms/step - loss: 67.5428 - mae: 6.1467 - val_loss: 687.8332 - val_mae: 23.0501
Epoch 6/30
593/593 [==============================] - 149s 251ms/step - loss: 60.8773 - mae: 5.8539 - val_loss: 820.5236 - val_mae: 25.2199
Epoch 7/30
593/593 [==============================] - 149s 251ms/step - loss: 55.4544 - mae: 5.5928 - val_loss: 538.6665 - val_mae: 19.8335
Epoch 8/30
593/5

In [23]:
model.save('age_detection_model.h5')